<h1 align="center"><font color="yellow">Sincronizando fontes de dados com armazenamentos de vetores</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [Syncing data sources to vector stores](https://blog.langchain.dev/syncing-data-sources-to-vector-stores/?ref=langchain-blog-newsletter)

* [LangChain Indexing API](https://python.langchain.com/docs/modules/data_connection/indexing?ref=blog.langchain.dev)

# <font color="red">Contextualizando</font>

<font color="orange">Os aplicativos LLM mais complexos e com uso intensivo de conhecimento exigem recuperação de dados em tempo de execução para `Retrieval Augmented Generation` (RAG). Um componente central da pilha RAG típica é um armazenamento de vetores, que é usado para potencializar a recuperação de documentos.</font>

![Alt text](image.png)

<font color="orange">O uso de um armazenamento de vetores requer a configuração de um `pipeline de indexação` para carregar dados de fontes (`um site, um arquivo, etc.`), transformar os dados em documentos, incorporar esses documentos e inserir os `Embeddings` e documentos no armazenamento de vetores.

Se suas fontes de dados ou etapas de processamento mudarem, os dados precisarão ser reindexados. Se isso acontecer regularmente e as alterações forem incrementais, torna-se valioso desduplicar o conteúdo que está sendo indexado com o conteúdo já no armazenamento de vetores. Isso evita gastar tempo e dinheiro com trabalho redundante. Também se torna importante configurar processos de limpeza de armazenamento de vetores para remover dados obsoletos de seu armazenamento de vetores.</font>

# <font color="red">API de indexação LangChain</font>

A nova `API de indexação LangChain` facilita carregar e manter sincronizados (`sync`) documentos de qualquer fonte em um armazenamento de vetores. Especificamente, isso ajuda:

* Evitar escrever conteúdo duplicado no armazenamento de vetores

* Evitar reescrever conteúdo inalterado

* Evitar recalcular `Embeddings` sobre conteúdo inalterado


Fundamentalmente, a `API de indexação` funcionará mesmo com documentos que passaram por diversas etapas de transformação (`por exemplo`, através de chunking de texto) em relação aos documentos de origem originais.

# <font color="red">Como funciona</font>

A `indexação LangChain` utiliza um gerenciador de registros (`RecordManager`) que monitora as gravações de documentos em um armazenamento de vetores (`Vector Store`).

Ao indexar o conteúdo, os `hashes` são calculados para cada documento e as seguintes informações são armazenadas no gerenciador de registros:

* o hash do documento (hash do conteúdo da página e dos metadados)

* hora de escrever

* o ID da fonte – cada documento deve incluir informações em seus metadados para nos permitir determinar a fonte final deste documento

# <font color="red">Modos de limpeza</font>

Ao `reindexar documentos em um armazenamento de vetores`, é possível que alguns documentos existentes no armazenamento de vetores sejam excluídos. Se você fez alterações na forma como os documentos são processados ​​antes da inserção ou os documentos de origem foram alterados, você desejará remover quaisquer documentos existentes que venham da mesma origem dos novos documentos que estão sendo indexados. Se alguns documentos de origem foram excluídos, você desejará excluir todos os documentos existentes no armazenamento de vetores e substituí-los pelos documentos reindexados.

Os modos de limpeza da `API de indexação` permitem escolher o comportamento desejado:

![Alt text](image-1.png)

# <font color="red">Vendo isso em ação</font>

<font color="orange">`Primeiro` vamos inicializar nosso armazenamento de vetores. Faremos uma demonstração com o `ElasticsearchStore`, pois ele atende aos pré-requisitos de suporte à `inserção` (insertion) e `exclusão` (deletion). Consulte a seção Documentos de [requisitos](https://python.langchain.com/docs/modules/data_connection/indexing?ref=blog.langchain.dev#requirements) para obter mais informações sobre os requisitos do armazenamento de vetores.</font>

In [7]:
%pip show langchain

Name: langchain
Version: 0.0.288
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/venv_LLMs/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental, ragas
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [2]:
# # !pip install openai elasticsearch

# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import ElasticsearchStore

# collection_name = "eddy_test_index"

# # Set env var OPENAI_API_KEY:
# embedding = OpenAIEmbeddings()

# # Execute uma instância do Elasticsearch localmente:
# # !docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.9.0
# vector_store = ElasticsearchStore(es_url="http://localhost:9200",
#                                   index_name=collection_name,
#                                   embedding=embedding
#                                  )


from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from langchain.document_loaders import TextLoader

embeddings = OpenAIEmbeddings()

import psycopg2

# Constrói a string de conexão PGVector a partir dos parâmetros.
host= os.environ['DB_HOST']
port= os.environ['DB_PORT']
user= os.environ['DB_USER']
password= os.environ['DB_PASSWORD']
dbname= os.environ['DB_NAME']

CONNECTION_STRING = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
COLLECTION_NAME = "Eddy_vectordb"


vectorstore = PGVector.from_documents(
    [],
    embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING
)



<font color="orange">E agora vamos inicializar e criar um esquema para nosso gerenciador de registros, para o qual usaremos apenas uma tabela `SQLite`:</font>

In [3]:
# from langchain.indexes import SQLRecordManager

# namespace = f"elasticsearch/{collection_name}"

# record_manager = SQLRecordManager(namespace, db_url="sqlite:///eddy_record_manager_cache.sql")
# record_manager.create_schema()


from langchain.indexes import SQLRecordManager

# Atualize o namespace para refletir PGVector:
namespace = f"pgvector/{COLLECTION_NAME}"

record_manager = SQLRecordManager(namespace,
                                  db_url=CONNECTION_STRING
                                 )


# Criar schema para o Gerenciador de registros (record):
record_manager.create_schema()


In [4]:
from langchain.schema import Document
from langchain.indexes import SQLRecordManager, index


doc1 = Document(page_content="Meu gatinho Schrödinger", metadata={"source": "kitty.txt"})
doc2 = Document(page_content="O cachorro de chama Rabito.", metadata={"source": "doggy.txt"})


def _clear():
    """Método auxiliar hacky para limpar o conteúdo. Veja a seção do modo `full` para entender por que funciona."""
    index([],
          record_manager,
          vectorstore,
          cleanup="full",
          source_id_key="source")


In [5]:
_clear()

In [6]:
index(
    [doc1, doc2],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source"
     )


{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}